In [11]:
import os
import pathlib

# Application packages
from tabulate import tabulate
import netCDF4

# stage_in packages
from unity_sds_client.resources.collection import Collection

# stage_out packages
from datetime import datetime, timezone
from unity_sds_client.resources.dataset import Dataset
from unity_sds_client.resources.data_file import DataFile

In [12]:
input_stac_collection_file = 'test/stage_in/stage_in_results.json' # type: stage-in
output_stac_catalog_dir    = 'test/process_results/'                    # type: stage-out

# Filename written to the working directory
summary_table_filename = "summary_table.txt"

#For eventual catalogging of this file in the unity environment
output_collection="example-app-collection___1"


# Examples of optional arbitrary arguments of different data types
example_argument_int = 1
example_argument_float = 1.0
example_argument_string = "string"
example_argument_bool = True
example_argument_empty = None # type: string Allow a null value or a string


# Output Example Arguments

Table useful for debugging connection of arguments from CWL into notebook. Used for unity-app-generator development and as an example, not an application requirement.

In [13]:
argument_names = [
    'example_argument_int',
    'example_argument_float',
    'example_argument_string',
    'example_argument_bool',
    'example_argument_empty',
]

table_data = []
column_names = ['argument_name', 'type', 'value']
for arg_name in argument_names:
    arg_val = locals()[arg_name]
    table_data.append( (arg_name, type(arg_val), arg_val) )

tabulate(table_data, headers=column_names, tablefmt='html')

argument_name,type,value
example_argument_int,<class 'int'>,1
example_argument_float,<class 'float'>,1.0
example_argument_string,<class 'str'>,string
example_argument_bool,<class 'bool'>,True
example_argument_empty,<class 'NoneType'>,


# Import Files from STAC Item Collection

Load filenames from the stage_in STAC item collection file

In [14]:
inp_collection = Collection.from_stac(input_stac_collection_file)
data_filenames = inp_collection.data_locations()

data_filenames

['/home/jovyan/unity-tutorial-application/test/stage_in/./SNDR.SS1330.CHIRP.20160822T0005.m06.g001.L1_AQ.std.v02_48.G.200425095850.nc',
 '/home/jovyan/unity-tutorial-application/test/stage_in/./SNDR.SS1330.CHIRP.20160822T0011.m06.g002.L1_AQ.std.v02_48.G.200425095901.nc']

# Summary Table

AKA the result of this "application"

In [15]:
column_names = [
    'product_name',
    'product_name_type_id',
    'shortname',
    'product_version',
    'date_created',
    'time_coverage_start',
    'time_coverage_end',
    'geospatial_lat_mid',
    'geospatial_lon_mid',
]

In [17]:
table_data = []
for data_file in data_filenames:
    nc_file = netCDF4.Dataset(data_file, "r")
    
    table_row = []
    for col_name in column_names:
        table_row.append(getattr(nc_file, col_name))
    table_data.append(table_row)

OSError: [Errno -51] NetCDF: Unknown file format: '/home/jovyan/unity-tutorial-application/test/stage_in/./SNDR.SS1330.CHIRP.20160822T0005.m06.g001.L1_AQ.std.v02_48.G.200425095850.nc'

In [18]:
# Output the table in html format
tabulate(table_data, headers=column_names, tablefmt='html')

'<table>\n<thead>\n<tr></tr>\n</thead>\n<tbody>\n</tbody>\n</table>'

In [19]:
# Write the table in text format
pathlib.Path(output_stac_catalog_dir).mkdir(parents=True, exist_ok=True)
output_filename = os.path.join(output_stac_catalog_dir, summary_table_filename)
with open(output_filename, "w") as summary_file:
    summary_file.write(tabulate(table_data, headers=column_names))

# Create stage-out item catalog

In [9]:
# Create a collection
out_collection  = Collection(output_collection)
    
# Create a Dataset for the collection
dataset = Dataset(
    name=os.path.splitext(summary_table_filename)[0], 
    collection_id=out_collection.collection_id, 
    start_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(), 
    end_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(),
    creation_time=datetime.utcnow().replace(tzinfo=timezone.utc).isoformat(),
)

# Add output file(s) to the dataset
dataset.add_data_file(DataFile("data", summary_table_filename))

#when we run "to_stac" below, this file  will be generated. this needs to be added to the stac file itself for future reference.
dataset.add_data_file(DataFile("metadata", output_stac_catalog_dir + summary_table_filename +'.json' ))


# Add the dataset to the collection
#out_collection.add_dataset(dataset)
out_collection._datasets.append(dataset)

Collection.to_stac(out_collection, output_stac_catalog_dir)